In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import cv2
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from GazeTracking.gaze_tracking import GazeTracking
import xlsxwriter     
import time      

# Define the CircumplexModel class
class CircumplexModel:
    def __init__(self):
        # Define the emotions and their corresponding ranges
        self.emotions = {
            1: (0.81, 0.88),  # Fear & Angry
            2: (0.775, 0.798),  # Disgust
            3: (0.946, 0.986),  # Happy 
            4: (0.91, 0.943), #Neutral
            5: (0.88, 0.91),   # Sad
            6: (0.9863, 1.067)  # Surprise
        }
        # Define the engagement ranges for each emotion
        self.engagement_ranges = {
            1: (0.81, 0.88),  # Fear & Angry
            2: (0.775, 0.798),  # Disgust
            3: (0.946, 0.986),  # Happy 
            4: (0.91, 0.943), #Neutral
            5: (0.88, 0.91),   # Sad
            6: (0.9863, 1.067)  # Surprise
        }
        # Define the outer labels for each quadrant
        self.outer_labels = {
            1: "High Negative",
            2: "Disengagement",
            3: "Pleasantness",
            4: "Low Negative",
            5: "Unpleasantness",
            6: "Engagement"
        }

    # Function to calculate the engagement level
    def calculate_engagement(self, hr, vr):
        return np.sqrt(hr**2 + vr**2)

    # Function to get the emotion based on the engagement level
    def get_emotion(self, engagement):
        for emotion, range in self.emotions.items():
            if range[0] <= engagement <= range[1]:
                return emotion
        return None

    # Function to get the engagement quadrant based on the engagement level
    def get_engagement_quadrant(self, engagement):
        for quadrant, range in self.engagement_ranges.items():
            if range[0] <= engagement <= range[1]:
                return quadrant
        return None

    # Function to get the outer label based on the angular bisector
    def get_outer_label(self, emotion, engagement_quadrant):
        # Calculate the angular bisector
        bisector = (emotion + engagement_quadrant) / 2

        # Round to the nearest integer to get the quadrant
        quadrant = round(bisector)

        # Get the outer label
        return self.outer_labels[quadrant]

# Initialize CircumplexModel
model = CircumplexModel()

# Open the webcam
webcam = cv2.VideoCapture(0)
print("Opened camera")

# Load the emotion detection model
emotion_model = load_model(r'EmotionRecognitionmodels/face.hdf5')
print("Emotion detection model loaded")

class_labels=['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise'] #classes of the emotions

cascade_classifier=cv2.CascadeClassifier(r'EmotionRecognitionmodels/frontalface.xml') #model to search for face in an image

# Initialize gaze tracking
gaze = GazeTracking()
print("Gaze tracking object created")

# Create an Excel workbook
book = xlsxwriter.Workbook(r'Data/sheets/data2.xlsx')
sheet = book.add_worksheet('data')
print("Excel sheet opened")

# Initialize variables
row = 0
column = 0
TT = 240  # Number of frames
sleep_time = 0.499  # Sleep time between frames

# Process each frame
while TT > 0:
    _, frame = webcam.read()

    # Detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cascade_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_CUBIC)
        crop = frame[y:y+h, x:x+w]

        if np.sum([roi_gray]) != 0:
            # Emotion Detection
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            preds = emotion_model.predict(roi)[0]
            label = class_labels[preds.argmax()]

            # Gaze Tracking
            gaze.refresh(frame)
            hr = gaze.horizontal_ratio()
            vr = gaze.vertical_ratio()
            engagement = model.calculate_engagement(hr, vr)
            engagement_quadrant = model.get_engagement_quadrant(engagement)
            outer_label = model.get_outer_label(preds.argmax(), engagement_quadrant)
            if not hr and not vr:
                hr, vr = 0.0, 0.0

            # Write data to Excel
            data_row = [hr, vr, preds.argmax(), engagement, outer_label]
            for data_item in data_row:
                sheet.write(row, column, data_item)
                column += 1
            row += 1
            column = 0

            # Save image
            cv2.imwrite(r'Data/Images/kang' + str(i) + '.jpg', crop)
            sheet.insert_image(row - 1, 6, r'Data/Images/kang' + str(i) + '.jpg')
            i += 1

    if cv2.waitKey(1) == 27:
        break
    time.sleep(sleep_time)
    TT -= 1

# Close workbook
book.close()



Opened camera


Emotion detection model loaded
Gaze tracking object created
Excel sheet opened
1/1 [==============================] - 1s 670ms/step


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'